# Few-Shots Prompting

Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

## References:
* [Touvron et al. 2023](https://arxiv.org/pdf/2302.13971.pdf): present few shot properties  when models were scaled to a sufficient size
* [Kaplan et al., 2020](https://arxiv.org/abs/2001.08361)
* [Brown et al. 2020](https://arxiv.org/abs/2005.14165)


## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Ffew_shots.ipynb)



In [1]:
##
## FEW SHOTS PROMPTING
##

from _pipeline import create_payload, model_req

#### (1) Adjust the inbounding  Prompt, simulating inbounding requests from users or other systems
MESSAGE = "Calculate 984 * log(2)"

#### (2) Adjust the Prompt Engineering Technique to be applied, simulating Workflow Templates
FEW_SHOT = "You are a math teacher. If student asked 1 + 1 you answer 2. If student ask 987 * 2 you answer only 1974. Student asked; provide the result only: "
PROMPT = FEW_SHOT + '\n' + MESSAGE 

#### (3) Configure the Model request, simulating Workflow Orchestration
# Documentation: https://github.com/ollama/ollama/blob/main/docs/api.md
payload = create_payload(target="ollama",
                         model="llama3.2:latest", 
                         prompt=PROMPT, 
                         temperature=1.0, 
                         num_ctx=100, 
                         num_predict=100)

### YOU DONT NEED TO CONFIGURE ANYTHING ELSE FROM THIS POINT
# Send out to the model
time, response = model_req(payload=payload)
print(response)
if time: print(f'Time taken: {time}s')

!!ERROR!! Problem loading prompt-eng/_config
Time taken: -1s


In [2]:
import openai
import time

# Set your OpenAI API key (using your provided sample key)
openai.api_key = "sk-n6UrvoaItfPkH35KyG1D0e78rIeDGUb9_g5jgngRHeT3BlbkFGEGJfnh39Ny3CT2myPz7hhdXwSx2qlfNduTuqN0dkA"

# Inbound message for the prompt
MESSAGE = "Calculate 984 * log(2)"

# Few-shot prompt: providing examples as a math teacher
FEW_SHOT = ("You are a math teacher. "
            "If a student asks '1 + 1', you answer '2'. "
            "If a student asks '987 * 2', you answer only '1974'. "
            "Student asked; provide the result only: ")

# Combine the few-shot instructions with the inbound message
PROMPT = FEW_SHOT + "\n" + MESSAGE

def run_few_shot(prompt_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful math teacher who provides concise answers."},
            {"role": "user", "content": prompt_text}
        ],
        max_tokens=100,
        temperature=1.0,
    )
    return response.choices[0].message['content'].strip()

# Measure the time taken for the API call
start_time = time.time()
result = run_few_shot(PROMPT)
end_time = time.time()
elapsed = end_time - start_time

print("Response:")
print(result)
print(f"Time taken: {elapsed:.2f}s")


Response:
1968
Time taken: 0.71s


## How to improve it?

Following the findings from [Min et al. (2022)](https://arxiv.org/abs/2202.12837), here are a few more tips about demonstrations/exemplars when doing few-shot:

* "the label space and the distribution of the input text specified by the demonstrations are both important (regardless of whether the labels are correct for individual inputs)"
* the format you use also plays a key role in performance, even if you just use random labels, this is much better than no labels at all.
* additional results show that selecting random labels from a true distribution of labels (instead of a uniform distribution) also helps.